In [2]:
!pip install tensorflow matplotlib numpy
print("Libraries installed!")


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import shutil
import random

print("TensorFlow version:", tf.__version__)
print("✅ Cell 1 done!")

TensorFlow version: 2.20.0
✅ Cell 1 done!


In [4]:
# Download the dataset
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O cats_and_dogs_filtered.zip

# Extract the dataset
with zipfile.ZipFile('cats_and_dogs_filtered.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

# Set paths
base_dir = 'cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = 'test_data'

# Create test directory
if not os.path.exists(test_dir):
    os.makedirs(test_dir)
    
    # Copy some images for testing
    test_cats = os.path.join(test_dir, 'cats')
    test_dogs = os.path.join(test_dir, 'dogs')
    os.makedirs(test_cats, exist_ok=True)
    os.makedirs(test_dogs, exist_ok=True)
    
    # Copy 25 cats and 25 dogs
    cat_files = os.listdir(os.path.join(train_dir, 'cats'))[:25]
    dog_files = os.listdir(os.path.join(train_dir, 'dogs'))[:25]
    
    for f in cat_files:
        shutil.copy(os.path.join(train_dir, 'cats', f), os.path.join(test_cats, f))
    for f in dog_files:
        shutil.copy(os.path.join(train_dir, 'dogs', f), os.path.join(test_dogs, f))

print("✅ Dataset ready!")
print("Train images:", len(os.listdir(os.path.join(train_dir, 'cats'))) * 2)

'wget' is not recognized as an internal or external command,
operable program or batch file.


FileNotFoundError: [Errno 2] No such file or directory: 'cats_and_dogs_filtered.zip'

In [ ]:
print("Creating image generators...")

IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32

# Create generators
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)
test_image_generator = ImageDataGenerator(rescale=1./255)

# Flow from directories
train_data_gen = train_image_generator.flow_from_directory(
    directory=train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_data_gen = validation_image_generator.flow_from_directory(
    directory=validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_data_gen = test_image_generator.flow_from_directory(
    directory=test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print("✅ Generators created!")

In [ ]:
def plotImages(images_arr, probabilities=None):
    fig, axes = plt.subplots(1, 5, figsize=(20, 20))
    axes = axes.flatten()
    
    for i, img in enumerate(images_arr[:5]):
        axes[i].imshow(img)
        if probabilities is not None and i < len(probabilities):
            axes[i].set_title(f"Prob: {probabilities[i]:.2f}")
        axes[i].axis('off')
    plt.tight_layout()
    plt.show()

# Test the function
sample_images, _ = next(train_data_gen)
plotImages(sample_images)
print("✅ Plot function ready!")

In [ ]:
print("Creating augmented generator...")

train_image_generator_augmented = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_data_gen_augmented = train_image_generator_augmented.flow_from_directory(
    directory=train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

print("✅ Augmented generator ready!")

In [ ]:
print("Building CNN model...")

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("✅ Model built!")
model.summary()

In [ ]:
print("Starting training... (This takes 5-10 minutes)")

epochs = 20

history = model.fit(
    train_data_gen_augmented,
    steps_per_epoch=len(train_data_gen) // 3,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=len(val_data_gen) // 3,
    verbose=1
)

print("✅ Training complete!")

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
print("Evaluating on test data...")

test_loss, test_accuracy = model.evaluate(test_data_gen)
print(f"\n📊 Test Accuracy: {test_accuracy*100:.2f}%")

if test_accuracy >= 0.63:
    print("🎉 SUCCESS! You passed the challenge!")
    if test_accuracy >= 0.70:
        print("🌟 EXTRA CREDIT! Over 70% accuracy!")
else:
    print(f"📝 Need 63%. Current: {test_accuracy*100:.2f}%. Try running again or add more epochs.")

In [ ]:
print("Making predictions on sample images...")

test_data_gen.reset()
test_images, test_labels = next(test_data_gen)

predictions = model.predict(test_images)
predictions = predictions.flatten()

# Show results
plt.figure(figsize=(15, 10))
for i in range(min(10, len(test_images))):
    plt.subplot(2, 5, i+1)
    plt.imshow(test_images[i])
    
    pred_class = "Dog" if predictions[i] > 0.5 else "Cat"
    actual_class = "Dog" if test_labels[i] == 1 else "Cat"
    color = 'green' if pred_class == actual_class else 'red'
    
    plt.title(f"Pred: {pred_class}\nActual: {actual_class}", color=color)
    plt.axis('off')
plt.tight_layout()
plt.show()

correct = sum(1 for i in range(len(predictions)) 
              if (predictions[i] > 0.5 and test_labels[i] == 1) or 
                 (predictions[i] <= 0.5 and test_labels[i] == 0))
print(f"Sample accuracy: {correct/len(predictions)*100:.2f}%")
print("\n✅ Project Complete!")